Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

Задания:

1) Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape(). Всю предобработку делаем в новом столбце 'clean_tweet'

2) Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function).

3) Изменим регистр твитов на нижний с помощью .lower().

4) Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

5) Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

6) Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

7) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

8) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

9) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

10) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

11) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

12) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

13) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

14) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

15) Сохраним результат предобработки в pickle-файл.

In [1]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os
import html
from copy import copy

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...


In [3]:
train_df = pd.read_csv('train_tweets.csv')
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
test_df = pd.read_csv('test_tweets.csv')
test_df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [5]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head()

C:\Users\nikit\AppData\Local\Temp\ipykernel_16744\539692593.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combine_df = train_df.append(test_df, ignore_index = True, sort = False)


,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [6]:
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   label   31962 non-null  float64
 2   tweet   49159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB
None


In [7]:
combine_df

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation
...,...,...,...
49154,49155,NaN,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...
49157,49158,NaN,"happy, at work conference: right mindset leads..."


1. Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape()

In [8]:
df = combine_df.copy()
df['tweet'] = df['tweet'].apply(html.unescape)
df

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation
...,...,...,...
49154,49155,NaN,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) & u...
49157,49158,NaN,"happy, at work conference: right mindset leads..."


2. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function)


In [9]:
def del_user(input_text):
    return re.sub('@[\w]*', ' ', input_text)
df['tweet'] = df['tweet'].apply(del_user)
df

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so se...
1,2,0.0,thanks for #lyft credit i can't use cause ...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation
...,...,...,...
49154,49155,NaN,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) & u...
49157,49158,NaN,"happy, at work conference: right mindset leads..."


3. Изменим регистр твитов на нижний с помощью .lower()

In [10]:
df['tweet'] = [txt.lower() for txt in df['tweet']]
df

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so se...
1,2,0.0,thanks for #lyft credit i can't use cause ...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation
...,...,...,...
49154,49155,NaN,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) & u...
49157,49158,NaN,"happy, at work conference: right mindset leads..."


4. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [11]:
def replace_word(text, dictionary):
    regexp = "|".join(re.escape(k) for k in dictionary)
    return re.sub(regexp, lambda m: dictionary[m.group(0)], text)
df['tweet'] = [replace_word(text, apostrophe_dict) for text in df.tweet]
df

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so se...
1,2,0.0,thanks for #lyft credit i cannot use cause...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation
...,...,...,...
49154,49155,NaN,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) & u...
49157,49158,NaN,"happy, at work conference: right mindset leads..."


5. Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [12]:
df['tweet'] = [replace_word(text, short_word_dict) for text in df.tweet]
df

,id,label,tweet
0,1,0.0,when a fatheare is dysfyounctional and is s...
1,2,0.0,thanks foare #lyft careedit i cannot youse...
2,3,0.0,bihday yoyouare majesty
3,4,0.0,#model i love you take with you all the time...
4,5,0.0,factsgrinyouide: society now #motivation
...,...,...,...
49154,49155,NaN,thoyougrinht factoarey: left-areigrinht polaar...
49155,49156,NaN,feelingrin like a mearemaid ð #haiareflip #...
49156,49157,NaN,#hillaarey #campaigrinned today in #ohio((oh m...
49157,49158,NaN,"happy, at woarek confeareence: areigrinht mind..."


6. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [13]:
df['tweet'] = [replace_word(text, emoticon_dict) for text in df.tweet]
df

,id,label,tweet
0,1,0.0,when a fatheare is dysfyounctional and is s...
1,2,0.0,thanks foare #lyft careedit i cannot youse...
2,3,0.0,bihday yoyouare majesty
3,4,0.0,#model i love you take with you all the time...
4,5,0.0,factsgrinyouide: society now #motivation
...,...,...,...
49154,49155,NaN,thoyougrinht factoarey: left-areigrinht polaar...
49155,49156,NaN,feelingrin like a mearemaid ð #haiareflip #...
49156,49157,NaN,#hillaarey #campaigrinned today in #ohio((oh m...
49157,49158,NaN,"happy, at woarek confeareence: areigrinht mind..."


7. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

In [14]:
df['tweet'] = [re.sub(r'[^\w\s]', ' ', text) for text in df.tweet]
df

,id,label,tweet
0,1,0.0,when a fatheare is dysfyounctional and is s...
1,2,0.0,thanks foare lyft careedit i cannot youse...
2,3,0.0,bihday yoyouare majesty
3,4,0.0,model i love you take with you all the time...
4,5,0.0,factsgrinyouide society now motivation
...,...,...,...
49154,49155,NaN,thoyougrinht factoarey left areigrinht polaar...
49155,49156,NaN,feelingrin like a mearemaid ð haiareflip ...
49156,49157,NaN,hillaarey campaigrinned today in ohio oh m...
49157,49158,NaN,happy at woarek confeareence areigrinht mind...


8. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

In [15]:
df['tweet'] = [re.sub(r'[^a-zA-Z0-9]', ' ', text) for text in df.tweet]
df

,id,label,tweet
0,1,0.0,when a fatheare is dysfyounctional and is s...
1,2,0.0,thanks foare lyft careedit i cannot youse...
2,3,0.0,bihday yoyouare majesty
3,4,0.0,model i love you take with you all the time...
4,5,0.0,factsgrinyouide society now motivation
...,...,...,...
49154,49155,NaN,thoyougrinht factoarey left areigrinht polaar...
49155,49156,NaN,feelingrin like a mearemaid haiareflip ...
49156,49157,NaN,hillaarey campaigrinned today in ohio oh m...
49157,49158,NaN,happy at woarek confeareence areigrinht mind...


9. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

In [16]:
df['tweet'] = [re.sub(r'[^a-zA-Z]', ' ', text) for text in df.tweet]
df

,id,label,tweet
0,1,0.0,when a fatheare is dysfyounctional and is s...
1,2,0.0,thanks foare lyft careedit i cannot youse...
2,3,0.0,bihday yoyouare majesty
3,4,0.0,model i love you take with you all the time...
4,5,0.0,factsgrinyouide society now motivation
...,...,...,...
49154,49155,NaN,thoyougrinht factoarey left areigrinht polaar...
49155,49156,NaN,feelingrin like a mearemaid haiareflip ...
49156,49157,NaN,hillaarey campaigrinned today in ohio oh m...
49157,49158,NaN,happy at woarek confeareence areigrinht mind...


10. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

In [17]:
df['tweet'] = [' '.join([w for w in x.split() if len(w)>1]) for x in df.tweet]
df

,id,label,tweet
0,1,0.0,when fatheare is dysfyounctional and is so sel...
1,2,0.0,thanks foare lyft careedit cannot youse cayous...
2,3,0.0,bihday yoyouare majesty
3,4,0.0,model love you take with you all the time in y...
4,5,0.0,factsgrinyouide society now motivation
...,...,...,...
49154,49155,NaN,thoyougrinht factoarey left areigrinht polaare...
49155,49156,NaN,feelingrin like mearemaid haiareflip nevearear...
49156,49157,NaN,hillaarey campaigrinned today in ohio oh my go...
49157,49158,NaN,happy at woarek confeareence areigrinht mindse...


11. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [18]:
df['tweet_token'] = [nltk.tokenize.word_tokenize(text) for text in df.tweet]
df

,id,label,tweet,tweet_token
0,1,0.0,when fatheare is dysfyounctional and is so sel...,"[when, fatheare, is, dysfyounctional, and, is,..."
1,2,0.0,thanks foare lyft careedit cannot youse cayous...,"[thanks, foare, lyft, careedit, can, not, yous..."
2,3,0.0,bihday yoyouare majesty,"[bihday, yoyouare, majesty]"
3,4,0.0,model love you take with you all the time in y...,"[model, love, you, take, with, you, all, the, ..."
4,5,0.0,factsgrinyouide society now motivation,"[factsgrinyouide, society, now, motivation]"
...,...,...,...,...
49154,49155,NaN,thoyougrinht factoarey left areigrinht polaare...,"[thoyougrinht, factoarey, left, areigrinht, po..."
49155,49156,NaN,feelingrin like mearemaid haiareflip nevearear...,"[feelingrin, like, mearemaid, haiareflip, neve..."
49156,49157,NaN,hillaarey campaigrinned today in ohio oh my go...,"[hillaarey, campaigrinned, today, in, ohio, oh..."
49157,49158,NaN,happy at woarek confeareence areigrinht mindse...,"[happy, at, woarek, confeareence, areigrinht, ..."


12. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [19]:
stop_words = nltk.corpus.stopwords.words('english')
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [20]:
df['tweet_token_filtered'] = [[word for word in text if word not in stop_words] for text in df.tweet_token]
df

,id,label,tweet,tweet_token,tweet_token_filtered
0,1,0.0,when fatheare is dysfyounctional and is so sel...,"[when, fatheare, is, dysfyounctional, and, is,...","[fatheare, dysfyounctional, selfish, dareagrin..."
1,2,0.0,thanks foare lyft careedit cannot youse cayous...,"[thanks, foare, lyft, careedit, can, not, yous...","[thanks, foare, lyft, careedit, youse, cayouse..."
2,3,0.0,bihday yoyouare majesty,"[bihday, yoyouare, majesty]","[bihday, yoyouare, majesty]"
3,4,0.0,model love you take with you all the time in y...,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, youare]"
4,5,0.0,factsgrinyouide society now motivation,"[factsgrinyouide, society, now, motivation]","[factsgrinyouide, society, motivation]"
...,...,...,...,...,...
49154,49155,NaN,thoyougrinht factoarey left areigrinht polaare...,"[thoyougrinht, factoarey, left, areigrinht, po...","[thoyougrinht, factoarey, left, areigrinht, po..."
49155,49156,NaN,feelingrin like mearemaid haiareflip nevearear...,"[feelingrin, like, mearemaid, haiareflip, neve...","[feelingrin, like, mearemaid, haiareflip, neve..."
49156,49157,NaN,hillaarey campaigrinned today in ohio oh my go...,"[hillaarey, campaigrinned, today, in, ohio, oh...","[hillaarey, campaigrinned, today, ohio, oh, go..."
49157,49158,NaN,happy at woarek confeareence areigrinht mindse...,"[happy, at, woarek, confeareence, areigrinht, ...","[happy, woarek, confeareence, areigrinht, mind..."


13. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [21]:
stemmer = nltk.stem.porter.PorterStemmer()
df['tweet_stemmed'] = [[stemmer.stem(plural) for plural in plurals] for plurals in df.tweet_token_filtered]
df

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed
0,1,0.0,when fatheare is dysfyounctional and is so sel...,"[when, fatheare, is, dysfyounctional, and, is,...","[fatheare, dysfyounctional, selfish, dareagrin...","[fathear, dysfyounct, selfish, dareagrin, kid,..."
1,2,0.0,thanks foare lyft careedit cannot youse cayous...,"[thanks, foare, lyft, careedit, can, not, yous...","[thanks, foare, lyft, careedit, youse, cayouse...","[thank, foar, lyft, careedit, yous, cayous, of..."
2,3,0.0,bihday yoyouare majesty,"[bihday, yoyouare, majesty]","[bihday, yoyouare, majesty]","[bihday, yoyouar, majesti]"
3,4,0.0,model love you take with you all the time in y...,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, youare]","[model, love, take, time, youar]"
4,5,0.0,factsgrinyouide society now motivation,"[factsgrinyouide, society, now, motivation]","[factsgrinyouide, society, motivation]","[factsgrinyouid, societi, motiv]"
...,...,...,...,...,...,...
49154,49155,NaN,thoyougrinht factoarey left areigrinht polaare...,"[thoyougrinht, factoarey, left, areigrinht, po...","[thoyougrinht, factoarey, left, areigrinht, po...","[thoyougrinht, factoarey, left, areigrinht, po..."
49155,49156,NaN,feelingrin like mearemaid haiareflip nevearear...,"[feelingrin, like, mearemaid, haiareflip, neve...","[feelingrin, like, mearemaid, haiareflip, neve...","[feelingrin, like, mearemaid, haiareflip, neve..."
49156,49157,NaN,hillaarey campaigrinned today in ohio oh my go...,"[hillaarey, campaigrinned, today, in, ohio, oh...","[hillaarey, campaigrinned, today, ohio, oh, go...","[hillaarey, campaigrin, today, ohio, oh, god, ..."
49157,49158,NaN,happy at woarek confeareence areigrinht mindse...,"[happy, at, woarek, confeareence, areigrinht, ...","[happy, woarek, confeareence, areigrinht, mind...","[happi, woarek, confear, areigrinht, mindset, ..."


14. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [22]:
lemmatizer = nltk.stem.WordNetLemmatizer()
df['tweet_lemmatized'] = [[lemmatizer.lemmatize(plural) for plural in plurals] for plurals in df.tweet_token_filtered]
df

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when fatheare is dysfyounctional and is so sel...,"[when, fatheare, is, dysfyounctional, and, is,...","[fatheare, dysfyounctional, selfish, dareagrin...","[fathear, dysfyounct, selfish, dareagrin, kid,...","[fatheare, dysfyounctional, selfish, dareagrin..."
1,2,0.0,thanks foare lyft careedit cannot youse cayous...,"[thanks, foare, lyft, careedit, can, not, yous...","[thanks, foare, lyft, careedit, youse, cayouse...","[thank, foar, lyft, careedit, yous, cayous, of...","[thanks, foare, lyft, careedit, youse, cayouse..."
2,3,0.0,bihday yoyouare majesty,"[bihday, yoyouare, majesty]","[bihday, yoyouare, majesty]","[bihday, yoyouar, majesti]","[bihday, yoyouare, majesty]"
3,4,0.0,model love you take with you all the time in y...,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, youare]","[model, love, take, time, youar]","[model, love, take, time, youare]"
4,5,0.0,factsgrinyouide society now motivation,"[factsgrinyouide, society, now, motivation]","[factsgrinyouide, society, motivation]","[factsgrinyouid, societi, motiv]","[factsgrinyouide, society, motivation]"
...,...,...,...,...,...,...,...
49154,49155,NaN,thoyougrinht factoarey left areigrinht polaare...,"[thoyougrinht, factoarey, left, areigrinht, po...","[thoyougrinht, factoarey, left, areigrinht, po...","[thoyougrinht, factoarey, left, areigrinht, po...","[thoyougrinht, factoarey, left, areigrinht, po..."
49155,49156,NaN,feelingrin like mearemaid haiareflip nevearear...,"[feelingrin, like, mearemaid, haiareflip, neve...","[feelingrin, like, mearemaid, haiareflip, neve...","[feelingrin, like, mearemaid, haiareflip, neve...","[feelingrin, like, mearemaid, haiareflip, neve..."
49156,49157,NaN,hillaarey campaigrinned today in ohio oh my go...,"[hillaarey, campaigrinned, today, in, ohio, oh...","[hillaarey, campaigrinned, today, ohio, oh, go...","[hillaarey, campaigrin, today, ohio, oh, god, ...","[hillaarey, campaigrinned, today, ohio, oh, go..."
49157,49158,NaN,happy at woarek confeareence areigrinht mindse...,"[happy, at, woarek, confeareence, areigrinht, ...","[happy, woarek, confeareence, areigrinht, mind...","[happi, woarek, confear, areigrinht, mindset, ...","[happy, woarek, confeareence, areigrinht, mind..."


15. Сохраним результат предобработки в pickle-файл.

In [23]:
df.to_pickle('dummy.pkl')